In [74]:
import requests
from datetime import datetime
import time

# Using open source github api for aircraft
BASE_URL = "https://api.adsbdb.com/v0/aircraft"
CALLSIGN_URL = "https://api.adsbdb.com/v0/callsign"

# Plane tail numbers for each celebrity
aircraft_info = {
    'N350XX': 'Tom Cruise',
    'N621MM': 'Taylor Swift',
    'N628TS': 'Elon Musk'
}

# Get general info about the plane based on tail number
def get_aircraft_data(tail_number):
    url = f"{BASE_URL}/{tail_number}"
    try:
        response = requests.get(url)
        data = response.json()

        if "response" in data and "aircraft" in data["response"]:
            aircraft = data["response"]["aircraft"]
            return (f"* Type: {aircraft.get('type', 'N/A')}\n - ICAO Type: {aircraft.get('icao_type', 'N/A')}\n - Manufacturer: {aircraft.get('manufacturer', 'N/A')}\n - Registered Owner: {aircraft.get('registered_owner', 'N/A')}\n - Country: {aircraft.get('registered_owner_country_name', 'N/A')}")
        else:
            return "No aircraft data found for this tail number"
    except requests.RequestException as e:
        return f"Error fetching data: {str(e)}"

# Get data on the location from the last flight of the plane based on tail number (doesn't always work but worked for Tom Cruise's plane)
def get_aircraft_location(tail_number):
    url = f"{CALLSIGN_URL}/{tail_number}"
    try:
        response = requests.get(url)
        data = response.json()

        if "response" in data and "flightroute" in data["response"]:
            flightroute = data["response"]["flightroute"]
            origin = flightroute.get("origin", {})
            destination = flightroute.get("destination", {})

            origin_name = origin.get("name")
            origin_iata = origin.get("iata_code")
            origin_lat = origin.get("latitude")
            origin_lon = origin.get("longitude")
            origin_location = f"{origin.get('municipality')}, {origin.get('country_name')}, ({origin_lat}, {origin_lon})"

            dest_name = destination.get("name")
            dest_iata = destination.get("iata_code")
            dest_lat = destination.get("latitude")
            dest_lon = destination.get("longitude")
            dest_location = f"{destination.get('municipality')}, {destination.get('country_name')}, ({dest_lat}, {dest_lon})"

            return (f"* Last known flight:\n - Origin: {origin_name} ({origin_iata}) - {origin_location}\n - Destination: {dest_name} ({dest_iata}) - {dest_location}")
        else:
            return "No flight route data found for this tail number"
    except requests.RequestException as e:
        return f"Error fetching data: {str(e)}"

for tail_number, celebrity in aircraft_info.items():
    print(f"\n{celebrity} ({tail_number}):")
    flight_info = get_aircraft_data(tail_number)
    print(flight_info)
    flight_location = get_aircraft_location(tail_number)
    print(flight_location)
    time.sleep(1)


Tom Cruise (N350XX):
* Type: Challenger 300
 - ICAO Type: CL30
 - Manufacturer: Bombardier
 - Registered Owner: A-OK Jets
 - Country: United States
* Last known flight:
 - Origin: Tri-Cities Regional TN/VA Airport (TRI) - Bristol/Johnson/Kingsport, United States, (36.475201, -82.407401)
 - Destination: Napa County Airport (APC) - Napa, United States, (38.2132, -122.280998)

Taylor Swift (N621MM):
* Type: Falcon 7X
 - ICAO Type: FA7X
 - Manufacturer: Dassault
 - Registered Owner: Island Jet Inc
 - Country: United States
No flight route data found for this tail number

Elon Musk (N628TS):
* Type: G650 ER
 - ICAO Type: G650
 - Manufacturer: Gulfstream Aerospace
 - Registered Owner: Falcon Landing LLC
 - Country: United States
No flight route data found for this tail number
